In [32]:
# https://blog.ayoungprogrammer.com/2016/04/determining-gender-of-name-with-80.html/
# https://www.kdnuggets.com/2019/01/solve-90-nlp-problems-step-by-step-guide.html

In [62]:
# https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
import random, string
import nltk
from pprint import pprint
import pandas as pd
import numpy as np
pd.set_option('display.expand_frame_repr', False)
# nltk.download('names')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

import matplotlib.pyplot as plt
# plt.rcParams['axes.labelsize'] = 10
# plt.rcParams['xtick.labelsize'] = 8
# plt.rcParams['ytick.labelsize'] = 8
import seaborn as sns
# sns.set_style('darkgrid')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

### Load Data

In [3]:
from nltk.corpus import names

In [4]:
labeled_names = ([(name, 'male') for name in names.words('male.txt')] 
                 + [(name, 'female') for name in names.words('female.txt')])
random.shuffle(labeled_names)
df_orig = pd.DataFrame(labeled_names, columns=['name', 'label'])
print(len(df_orig))
df_orig.head()

7944


,name,label
0,Gerladina,female
1,Goldy,female
2,Etti,female
3,Barnabe,male
4,Mayer,male


In [5]:
# male / female distribution
print('Cnt male names: {}'.format(len(df_orig[df_orig['label']=='male'])))
print('Cnt female names: {}'.format(len(df_orig[df_orig['label']=='female'])))

Cnt male names: 2943
Cnt female names: 5001


In [101]:
# max name length
max = 0
for n in df_orig['name']:
    if len(n) > max:
        max = len(n)
        print(n)
print(max)

Gerladina
Elisabetta
Constantinos
Jean-Christophe
15


#### Data Preprocessing

These actions were prompted by the CountVectorizer().get_feature_names() output

* Removing name E'Lane

In [135]:
# view compound names
# print(df[df['name'].str.contains(' ')].name)
# print(df[df['name'].str.contains("'")].name)
# print(df[df['name'].str.contains('-')].name)

df = df_orig.copy()
df = df[~df['name'].str.contains("'")]
print(len(df))

7943


### Feature Engineering

#### Character Order

In [ ]:
def get_char_order(name):
    print(name)
    name = name.strip()
    arr = np.zeros(15)
#     print(string.ascii_lowercase.index('a'))
    for ind, x in enumerate('geraldine'):
        print(ind+1, x)
        print(string.ascii_lowercase.index(x))
        arr[ind] = string.ascii_lowercase.index(x)+1   
#     print(arr)
    return arr

# columns = [i for i in np.arange(15)]
# pd.DataFrame([arr],columns=columns)

print(df['name'].apply(lambda x: get_char_order(x)))

# df = pd.concat([df.reset_index(drop=True), 
#                 pd.DataFrame([get_char_order(df['name'])]).reset_index(drop=True)], axis=1)
print(len(df))
df.head()


Gerladina
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Goldy
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Etti
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Barnabe
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Mayer
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Malissa
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Golda
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Neille
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Flossy
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Katee
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Kellsie
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Penni
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Cherilynn
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Robbin
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Lydia
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11
6 d
3
7 i
8
8 n
13
9 e
4
Sonnie
1 g
6
2 e
4
3 r
17
4 a
0
5 l
11


#### Compund Name Feature

In [7]:
def is_compound(name):
    name = name.strip()
    if (' ' in name) or ('-' in name):
        return 1
    return 0

df['is_compound'] = df['name'].apply(lambda x: is_compound(x))
print(len(df))
df.head()

7943


,name,label,is_compound
0,Gerladina,female,0
1,Goldy,female,0
2,Etti,female,0
3,Barnabe,male,0
4,Mayer,male,0


#### Suffix Last Letter Feature

In [8]:
def suffix1(name):
    name = name.strip()
    return name[-1:].lower()

df['suffix1'] = df['name'].apply(lambda x: suffix1(x))
print(len(df))
df.head()

7943


,name,label,is_compound,suffix1
0,Gerladina,female,0,a
1,Goldy,female,0,y
2,Etti,female,0,i
3,Barnabe,male,0,e
4,Mayer,male,0,r


#### Suffix 2nd Last Letter Feature

In [9]:
def suffix2(name):
    name = name.strip()
    return name[-2:-1].lower()

df['suffix2'] = df['name'].apply(lambda x: str(suffix2(x)))
print(len(df))
df.head()

7943


,name,label,is_compound,suffix1,suffix2
0,Gerladina,female,0,a,n
1,Goldy,female,0,y,d
2,Etti,female,0,i,t
3,Barnabe,male,0,e,b
4,Mayer,male,0,r,e


#### Name Length Feature

In [10]:
def name_length(name):
    name = name.strip()
    return len(name)

df['name_length'] = df['name'].apply(lambda x: name_length(x))
print(len(df))
df.head()

7943


,name,label,is_compound,suffix1,suffix2,name_length
0,Gerladina,female,0,a,n,9
1,Goldy,female,0,y,d,5
2,Etti,female,0,i,t,4
3,Barnabe,male,0,e,b,7
4,Mayer,male,0,r,e,5


#### Character Frequency Feature

In [11]:
# Test CounVectorizer
# corpus = ['Raoul', 'Anna']

# count_vectorizer = CountVectorizer(analyzer='char', min_df=1)
# X = count_vectorizer.fit_transform(corpus)
# print(count_vectorizer.get_feature_names())
# print(pd.DataFrame(X.toarray()))

# count_vectorizer = CountVectorizer(analyzer='char', min_df=0.6)
# X = count_vectorizer.fit_transform(corpus)
# print(count_vectorizer.get_feature_names())
# print(pd.DataFrame(X.toarray()))

In [12]:
corpus = df['name'].tolist()

# apply CountVectorizer()
count_vectorizer = CountVectorizer(analyzer='char', min_df=1)
X = count_vectorizer.fit_transform(corpus)
X = pd.DataFrame(X.toarray())

# inspect
print(count_vectorizer.get_feature_names())
X.head()
print(len(X), len(df))

# add the new columns to df
df = pd.concat([df.reset_index(drop=True), 
                X.reset_index(drop=True)], axis=1)

# inspect
print(len(df))
# print(df.columns)
df.head()

[' ', '-', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
7943 7943
7943


,name,label,is_compound,suffix1,suffix2,name_length,0,1,2,3,...,18,19,20,21,22,23,24,25,26,27
0,Gerladina,female,0,a,n,9,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0
1,Goldy,female,0,y,d,5,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Etti,female,0,i,t,4,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,Barnabe,male,0,e,b,7,0,0,2,2,...,0,1,0,0,0,0,0,0,0,0
4,Mayer,male,0,r,e,5,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


#### Inspect Null Values

In [13]:
null_columns=df.columns[df.isnull().any()]
# print(df[null_columns].isnull().sum())
print(df[df.isnull().any(axis=1)][null_columns].head())

Empty DataFrame
Columns: []
Index: []


#### Encoding Binary Categorical Features

In [14]:
gender_mapping = {'male': 1, 'female': 0}
df['label'] = df['label'].map(lambda x: gender_mapping[x])
df.head()

,name,label,is_compound,suffix1,suffix2,name_length,0,1,2,3,...,18,19,20,21,22,23,24,25,26,27
0,Gerladina,0,0,a,n,9,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0
1,Goldy,0,0,y,d,5,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Etti,0,0,i,t,4,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,Barnabe,1,0,e,b,7,0,0,2,2,...,0,1,0,0,0,0,0,0,0,0
4,Mayer,1,0,r,e,5,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


#### Encoding Multiclass Categorical Features

https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [15]:
# set 'name' to be the index
df.set_index('name', inplace=True)

# Convert categorical variable into dummy/indicator variables
df = pd.get_dummies(df, prefix_sep='_', columns=['suffix1', 'suffix2'], drop_first=True)
print(len(df))
df.head()

7943


,label,is_compound,name_length,0,1,2,3,4,5,6,...,suffix2_p,suffix2_r,suffix2_s,suffix2_t,suffix2_u,suffix2_v,suffix2_w,suffix2_x,suffix2_y,suffix2_z
name,,,,,,,,,,,,,,,,,,,,,
Gerladina,0,0,9,0,0,2,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Goldy,0,0,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Etti,0,0,4,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
Barnabe,1,0,7,0,0,2,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Mayer,1,0,5,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### Classify

#### Split Train/Test

In [16]:
tmp = df.copy()
y = tmp.pop('label')
X = tmp

# 1st pass to create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

# 2nd pass to divide train set into train and dev sets
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, random_state=0)

print('len X_train: {}'.format(len(X_train)))
print('len X_dev: {}'.format(len(X_dev)))
print('len X_test: {}'.format(len(X_test)))

len X_train: 4467
len X_dev: 1490
len X_test: 1986


#### Model

In [17]:
# Train the model using the training sets

nb = GaussianNB()
nb.fit(X_train.select_dtypes(include=[np.number]), 
       y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

#### Predict & Evaluate

In [18]:
# Accuracy
# nltk.classify.accuracy(classifier, dev_set)
nb.score(X_dev.select_dtypes(include=[np.number]), 
         y_dev)

0.7557046979865771

#### Test unseen data

In [19]:
name = 'Arlo'
name = 'Anna'
name = 'Laura'
name = 'Eileen'

# get test name features (transpose to accomodate next step)
name_features = pd.DataFrame(get_features(name)).T

name_features = pd.concat([name_features, 
                           pd.DataFrame(vectorizer.transform([name]).toarray())],
                         axis=1)
name_features

# # make a helper df of the test name features
# name_features_enc = (pd.concat([
#                  pd.DataFrame(le_last_letter.transform(name_features.loc[:, 0]), columns=['last_letter_enc']),
#                  pd.DataFrame(le_last_2letters.transform(name_features.loc[:, 1]), columns=['last_2letters_enc']),
#                 ], 
#                 axis=1))
# print(name_features_enc)

# # predict label (must use first row of helper df & pass as list NOT as Series hence the outer [])
# predicted_label = nb.predict([name_features_enc.iloc[0,:]])
# print("Predicted label for {}: {}".format(name, predicted_label[0]))


NameError: name 'get_features' is not defined

#### Error Analysis

In [ ]:
# Predict labels for X_dev and isolate records with incorrect pred.

# get predictons for X_dev
pred = nb.predict(X_dev[['last_letter_enc', 'last_2letters_enc']])

# make a df composed of X_dev cols
df = (pd.concat([X_dev.reset_index(drop=True), 
                  y_dev.reset_index(drop=True), 
                  pd.DataFrame(pred, columns=['predicted'])], axis=1))

# get all incorrect predictions
df_e = df[df['label_enc'] != df['predicted']]
# get all correct predictions
df_c= df[df['label_enc'] == df['predicted']]

In [ ]:
# get subset of incorrect predictions
print('incorrect predictions\n')
i = 'n'
for g in ['male', 'female']:
    res = df_e[(df_e['last_letter']==i) & (df_e['label']==g)]
    print('cnt {} ending in {}: {}'.format(g, i,len(res)))
    if len(res)>0:
        print(res.head())

In [ ]:
# get subset of correct predictions
print('correct predictions\n')
i = 'n'
for g in ['male', 'female']:
    res = df_c[(df_c['last_letter']==i) & (df_c['label']==g)]
    print('cnt {} ending in {}: {}'.format(g, i,len(res)))
    if len(res)>0:
        print(res.head())

### Appendix: Exploratory Data Analysis

In [ ]:
# sns.countplot(x=df.label, data=df) 
# plt.show()
pd.DataFrame(df.groupby('label').size(), columns=['count']).T

In [ ]:
sns.countplot(x=df.last_letter, hue=df.label, data=df) 
plt.show()

In [ ]:
df_agg = df.groupby(['last_2letters', 'label']).count().reset_index()
# df_agg = df_agg[df_agg['count']>=50]
df_agg.head()

# print(df_agg[(df_agg['label']=='male') & (df_agg['count']>=100)])
# print(df_agg[(df_agg['label']=='female') & (df_agg['count']>=100)])

# sns.barplot(x=df_agg['last_2letters'], y=df_agg['count'], hue=df_agg['label'], data=df_agg) 
# plt.show()

In [ ]:
# Most Informative Features
# classifier.show_most_informative_features(5)
# """
# Most Informative Features
#              last_letter = 'k'              male : female =     42.2 : 1.0
#              last_letter = 'a'            female : male   =     33.8 : 1.0
#              last_letter = 'v'              male : female =     17.2 : 1.0
#              last_letter = 'f'              male : female =     15.6 : 1.0
#              last_letter = 'p'              male : female =     11.6 : 1.0
# """